In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_csv('../Resources/elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv', delimiter = ";")
df.head()

,Melding ID,Meldingstidspunkt,Meldingsdato,Meldingsklokkeslett,Starttidspunkt,Startdato,Startklokkeslett,Startposisjon bredde,Startposisjon lengde,Hovedområde start (kode),...,Art - FDIR,Art - gruppe (kode),Art - gruppe,Rundvekt,Lengdegruppe (kode),Lengdegruppe,Bruttotonnasje 1969,Bruttotonnasje annen,Bredde,Fartøylengde
0,1497177,01.01.2018,01.01.2018,00:00,31.12.2017,31.12.2017,00:00,"-60,35","-46,133",NaN,...,Antarktisk krill,506.0,Antarktisk krill,706714.0,5.0,28 m og over,9432.0,NaN,"19,87","133,88"
1,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Hyse,202.0,Hyse,9594.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"
2,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Torsk,201.0,Torsk,8510.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"
3,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Blåkveite,301.0,Blåkveite,196.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"
4,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Sei,203.0,Sei,134.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"


In [3]:
df = df[['Art - FDIR','Rundvekt', 'Hovedområde start', 'Starttidspunkt', 'Redskap FDIR']].copy()
df

,Art - FDIR,Rundvekt,Hovedområde start,Starttidspunkt,Redskap FDIR
0,Antarktisk krill,706714.0,NaN,31.12.2017,Flytetrål
1,Hyse,9594.0,Bjørnøya,30.12.2017 23:21,Bunntrål
2,Torsk,8510.0,Bjørnøya,30.12.2017 23:21,Bunntrål
3,Blåkveite,196.0,Bjørnøya,30.12.2017 23:21,Bunntrål
4,Sei,134.0,Bjørnøya,30.12.2017 23:21,Bunntrål
...,...,...,...,...,...
305429,Gråsteinbit,145.0,Vest-Spitsbergen,31.12.2018 19:41,Bunntrål
305430,Uer (vanlig),136.0,Vest-Spitsbergen,31.12.2018 19:41,Bunntrål
305431,Flekksteinbit,132.0,Vest-Spitsbergen,31.12.2018 19:41,Bunntrål
305432,Snabeluer,102.0,Vest-Spitsbergen,31.12.2018 19:41,Bunntrål


In [4]:
df.dtypes

Art - FDIR            object
Rundvekt             float64
Hovedområde start     object
Starttidspunkt        object
Redskap FDIR          object
dtype: object

In [5]:
print(df.isnull().sum())

Art - FDIR           4982
Rundvekt             4978
Hovedområde start    4124
Starttidspunkt          0
Redskap FDIR          188
dtype: int64


In [6]:
df = df.dropna()
df

,Art - FDIR,Rundvekt,Hovedområde start,Starttidspunkt,Redskap FDIR
1,Hyse,9594.0,Bjørnøya,30.12.2017 23:21,Bunntrål
2,Torsk,8510.0,Bjørnøya,30.12.2017 23:21,Bunntrål
3,Blåkveite,196.0,Bjørnøya,30.12.2017 23:21,Bunntrål
4,Sei,134.0,Bjørnøya,30.12.2017 23:21,Bunntrål
5,Hyse,9118.0,Bjørnøya,31.12.2017 05:48,Bunntrål
...,...,...,...,...,...
305429,Gråsteinbit,145.0,Vest-Spitsbergen,31.12.2018 19:41,Bunntrål
305430,Uer (vanlig),136.0,Vest-Spitsbergen,31.12.2018 19:41,Bunntrål
305431,Flekksteinbit,132.0,Vest-Spitsbergen,31.12.2018 19:41,Bunntrål
305432,Snabeluer,102.0,Vest-Spitsbergen,31.12.2018 19:41,Bunntrål


In [7]:
df['Starttidspunkt'] = pd.to_datetime(df['Starttidspunkt'], format='mixed')
df['month'] = df['Starttidspunkt'].dt.month
df['day'] = df['Starttidspunkt'].dt.day
df['hour'] = df['Starttidspunkt'].dt.hour
df = df.drop(['Starttidspunkt'], axis=1)

df.head(20)
df.dtypes

/var/folders/zd/6gz2dx9n4z55z1g2cr06p0c00000gn/T/ipykernel_50202/4193018403.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Starttidspunkt'] = pd.to_datetime(df['Starttidspunkt'], format='mixed')
/var/folders/zd/6gz2dx9n4z55z1g2cr06p0c00000gn/T/ipykernel_50202/4193018403.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = df['Starttidspunkt'].dt.month
/var/folders/zd/6gz2dx9n4z55z1g2cr06p0c00000gn/T/ipykernel_50202/4193018403.py:3: SettingWithCopyWarning: 
A value is trying to be set o

Art - FDIR            object
Rundvekt             float64
Hovedområde start     object
Redskap FDIR          object
month                  int32
day                    int32
hour                   int32
dtype: object

In [8]:
df = pd.get_dummies(df, columns=['Hovedområde start'])
df = pd.get_dummies(df, columns=['Redskap FDIR'])
df

,Art - FDIR,Rundvekt,month,day,hour,Hovedområde start_Admiralityfeltet,Hovedområde start_Bjørnøya,Hovedområde start_Britvinfeltet,Hovedområde start_Danmarkstredet,Hovedområde start_Eigersundbanken,...,Redskap FDIR_Flytetrål par,Redskap FDIR_Harpun og lignende uspesifiserte typer,Redskap FDIR_Juksa/pilk,Redskap FDIR_Reketrål,Redskap FDIR_Settegarn,Redskap FDIR_Snurpenot/ringnot,Redskap FDIR_Snurrevad,Redskap FDIR_Teiner,Redskap FDIR_Udefinert garn,Redskap FDIR_Udefinert trål
1,Hyse,9594.0,12,30,23,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Torsk,8510.0,12,30,23,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Blåkveite,196.0,12,30,23,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Sei,134.0,12,30,23,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,Hyse,9118.0,12,31,5,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305429,Gråsteinbit,145.0,12,31,19,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
305430,Uer (vanlig),136.0,12,31,19,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
305431,Flekksteinbit,132.0,12,31,19,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
305432,Snabeluer,102.0,12,31,19,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
X = df.drop(['Art - FDIR'], axis=1)
y = df[['Art - FDIR']]


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
clf = KNeighborsClassifier(n_neighbors=1000)
clf.fit(X_train, y_train)

/Users/stalekarlstad/Desktop/v2024/MachineLearning/INFO284/venv/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=1000)

In [18]:
print("Test set predictions: {}".format(clf.predict(X_train)))
print("Test set accuracy: {:.2f}".format(clf.score(X_test, y_test)))

Test set predictions: ['Breiflabb' 'Torsk' 'Torsk' ... 'Sei' 'Torsk' 'Torsk']
Test set accuracy: 0.22
